In [1]:
from sudoku import Sudoku

In [2]:
# Unsolved 9x9 sudoku with 80 clues
#s = Sudoku("327456891918372546465981732532864179791523684846719253653148927279630418184297365")

# Unsolved 9x9 sudoku with 75 clues
s = Sudoku("192700658437865912560291347749358126053612479216947583625179830971483265084526791")

# Unsolved 9x9 sudoku with 17 clues
#s = Sudoku("600000900020000000000000000000000059100080000000300200005060407890040000004000000")

# Solved 9x9 sudoku with 17 clues
#s = Sudoku("683417925529638174471592386367124859142985763958376241215869437896743512734251698")


In [3]:
s.printBoard()
print("")
s.printBoard()


1 9 2 | 7     | 6 5 8 
4 3 7 | 8 6 5 | 9 1 2 
5 6   | 2 9 1 | 3 4 7 
---------------------
7 4 9 | 3 5 8 | 1 2 6 
  5 3 | 6 1 2 | 4 7 9 
2 1 6 | 9 4 7 | 5 8 3 
---------------------
6 2 5 | 1 7 9 | 8 3   
9 7 1 | 4 8 3 | 2 6 5 
  8 4 | 5 2 6 | 7 9 1 

1 9 2 | 7     | 6 5 8 
4 3 7 | 8 6 5 | 9 1 2 
5 6   | 2 9 1 | 3 4 7 
---------------------
7 4 9 | 3 5 8 | 1 2 6 
  5 3 | 6 1 2 | 4 7 9 
2 1 6 | 9 4 7 | 5 8 3 
---------------------
6 2 5 | 1 7 9 | 8 3   
9 7 1 | 4 8 3 | 2 6 5 
  8 4 | 5 2 6 | 7 9 1 


In [4]:
s.isValid()
s.printBoard()


1 9 2 | 7     | 6 5 8 
4 3 7 | 8 6 5 | 9 1 2 
5 6   | 2 9 1 | 3 4 7 
---------------------
7 4 9 | 3 5 8 | 1 2 6 
  5 3 | 6 1 2 | 4 7 9 
2 1 6 | 9 4 7 | 5 8 3 
---------------------
6 2 5 | 1 7 9 | 8 3   
9 7 1 | 4 8 3 | 2 6 5 
  8 4 | 5 2 6 | 7 9 1 


In [5]:
s.isSolved()
s.printBoard()


1 9 2 | 7     | 6 5 8 
4 3 7 | 8 6 5 | 9 1 2 
5 6   | 2 9 1 | 3 4 7 
---------------------
7 4 9 | 3 5 8 | 1 2 6 
  5 3 | 6 1 2 | 4 7 9 
2 1 6 | 9 4 7 | 5 8 3 
---------------------
6 2 5 | 1 7 9 | 8 3   
9 7 1 | 4 8 3 | 2 6 5 
  8 4 | 5 2 6 | 7 9 1 


In [6]:
s.getPossibleMoves()

{(0, 4): [3], (0, 5): [4], (2, 2): [8], (4, 0): [8], (6, 8): [4], (8, 0): [3]}

In [7]:
s.doMove((0,4),3)

In [8]:
s.printBoard()

1 9 2 | 7 3   | 6 5 8 
4 3 7 | 8 6 5 | 9 1 2 
5 6   | 2 9 1 | 3 4 7 
---------------------
7 4 9 | 3 5 8 | 1 2 6 
  5 3 | 6 1 2 | 4 7 9 
2 1 6 | 9 4 7 | 5 8 3 
---------------------
6 2 5 | 1 7 9 | 8 3   
9 7 1 | 4 8 3 | 2 6 5 
  8 4 | 5 2 6 | 7 9 1 
